## Import

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mlp
import mplfinance as ml
import cv2
from ta.volatility import BollingerBands
from ta.momentum import kama
import pickle
import gc

## Configure Data

In [ ]:
mlp.use('agg')
min_data = pd.read_csv("../csv/Binance_BTCUSDT_minute.csv",skiprows=1)
min_data["date"] = pd.to_datetime(min_data["date"])
min_data.index = min_data["date"]
min_data["volume"] = min_data["Volume USDT"]
min_data = min_data.drop(["unix","Volume USDT","Volume BTC","date"],axis=1)

## Add Indicators

In [ ]:
indicator_bb = BollingerBands(close=min_data["close"], window=20, window_dev=2)
min_data["kama"] = kama(min_data["close"], window=10, pow1=2, pow2=30, fillna=False)
min_data['bb_bbm'] = indicator_bb.bollinger_mavg()
min_data['bb_bbh'] = indicator_bb.bollinger_hband()
min_data['bb_bbl'] = indicator_bb.bollinger_lband()
min_data['accu'] = ta.volume.acc_dist_index(high=min_data['high'],low=min_data['low'],close = min_data['close'],volume = min_data['volume'])
min_data['ema'] = ta.trend.ema_indicator(close = min_data['close'],window = 100)
min_data['rsi'] = ta.momentum.rsi(close = min_data["close"],window=100)
min_data = min_data.dropna()

min_data = min_data.drop(["symbol"],axis=1)
min_data = min_data.drop(["tradecount"],axis=1)
min_data = min_data[min_data["volume"]>0] ## Drop inactive minutes

## Create Images and PickleFile

In [ ]:
data = []
count=0
for i in list(range(len(min_data)-105))[::15]:
    textfile = open("i_list.txt", "w")
    textfile.write(str(i) + "\n")
    textfile.close()
    try:
        apdict = ml.make_addplot(min_data[['bb_bbl','bb_bbh','kama']][i:i+100])
        save = dict(fname='../csv/pics/tsave_{}.jpg'.format(i),dpi=200,pad_inches=0,bbox_inches="tight")
        ml.plot(min_data[i:i+100],volume=True,addplot=apdict,mav=(10,20),style='classic',update_width_config=dict(ohlc_linewidth=2,ohlc_ticksize=0.5),savefig=save)
        image = cv2.imread('../csv/pics/tsave_{}.jpg'.format(i))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray =gray [:805, :1100]
        change = (min_data["close"][i+115]-min_data["close"][i+100])
        if (change > 0):
                y=1
        elif(change < 0):
                y=0
        else:
                continue

        data.append([gray,y])
    except:
        print("passed ", i)
        pass
    count = count +  1
    if(count%1000 == 0):
        with open('../csv/pickles/filename_{}.pickle'.format(count), 'wb') as handle:
            pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print("saved ", count)
        del data
        cv2.imwrite('{}.jpg'.format(i), gray)
        gc.collect()
        data=[]